In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics

import joblib

2024-02-26 04:29:03.162873: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 04:29:03.166164: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 04:29:03.202740: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 04:29:03.202793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 04:29:03.204512: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_train = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_train.csv', low_memory=False)
x_val = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_val.csv', low_memory=False)
y_val = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_val.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_test.csv', low_memory=False)

In [4]:
from sklearn import preprocessing
s1 = preprocessing.MinMaxScaler()
s2 = preprocessing.MinMaxScaler()
s3 = preprocessing.MinMaxScaler()

x_train_scaled = s1.fit_transform(x_train)
x_val_scaled = s2.fit_transform(x_val)
x_test_scaled = s3.fit_transform(x_test)

x_train = np.copy(x_train_scaled)
x_val = np.copy(x_val_scaled)
x_test = np.copy(x_test_scaled)

In [5]:
model = joblib.load("/home/jovyan/WUSTL-IIoT/dl/dl_new/dnn.joblib")

In [6]:
np.unique(y_train)

array([0, 1, 2, 3, 4])

In [7]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=5,
    input_shape=(48,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [8]:
attack = FastGradientMethod(estimator=classifier, eps=0.01)

x_test_adv = attack.generate(x=x_test)

In [9]:
calculate_performance_metrics(x_test_adv, y_test, model)

7466/7466 [==============================] - 4s 537us/step
Accuracy: 0.9996316357882575
AUC: 0.9999592618644184

macro
Precision: 0.8546620972836386
Recall: 0.9230980771564614
F1 Score: 0.8506926062079415

weighted
Precision: 0.9998586600158942
Recall: 0.9996316357882575
F1 Score: 0.9997090276077586

Mean FNR: 0.07690192284353858
Mean TNR: 0.9999261362349952
Mean FPR: 7.386376500496653e-05
Mean TPR: 0.9230980771564614


In [10]:
attack = FastGradientMethod(estimator=classifier, eps=0.05)

x_test_adv = attack.generate(x=x_test)

In [11]:
calculate_performance_metrics(x_test_adv, y_test, model)

7466/7466 [==============================] - 4s 541us/step
Accuracy: 0.9766298023391128
AUC: 0.9923102230548245

macro
Precision: 0.6058777159620214
Recall: 0.7058531787297637
F1 Score: 0.5322798911701231

weighted
Precision: 0.9916243950165409
Recall: 0.9766298023391128
F1 Score: 0.9818773561168236

Mean FNR: 0.29414682127023634
Mean TNR: 0.9755582257149078
Mean FPR: 0.024441774285092217
Mean TPR: 0.7058531787297637


In [9]:
attack = FastGradientMethod(estimator=classifier, eps=0.1)

x_test_adv = attack.generate(x=x_test)

In [10]:
calculate_performance_metrics(x_test_adv, y_test, model)

7466/7466 [==============================] - 4s 541us/step
Accuracy: 0.930923338384388
AUC: 0.9788152234946763

macro
Precision: 0.5702436619641901
Recall: 0.5563407250885272
F1 Score: 0.37853840180647874

weighted
Precision: 0.9755988342780268
Recall: 0.930923338384388
F1 Score: 0.9246290306672239

Mean FNR: 0.44365927491147283
Mean TNR: 0.9339077776490544
Mean FPR: 0.0660922223509455
Mean TPR: 0.5563407250885272


In [11]:
attack = FastGradientMethod(estimator=classifier, eps=0.15)

x_test_adv = attack.generate(x=x_test)

In [12]:
calculate_performance_metrics(x_test_adv, y_test, model)

7466/7466 [==============================] - 4s 543us/step
Accuracy: 0.92813549105461
AUC: 0.9491874368162823

macro
Precision: 0.5304962944493712
Recall: 0.5427358159149694
F1 Score: 0.3744458339563273

weighted
Precision: 0.98159583599841
Recall: 0.92813549105461
F1 Score: 0.9260631741435346

Mean FNR: 0.45726418408503056
Mean TNR: 0.9602606770339703
Mean FPR: 0.039739322966029667
Mean TPR: 0.5427358159149694


In [13]:
attack = FastGradientMethod(estimator=classifier, eps=0.2)

x_test_adv = attack.generate(x=x_test)

In [14]:
calculate_performance_metrics(x_test_adv, y_test, model)

7466/7466 [==============================] - 4s 540us/step
Accuracy: 0.9279429370348355
AUC: 0.8709985968075877

macro
Precision: 0.4965207205987987
Recall: 0.3727249462294309
F1 Score: 0.3383576788658599

weighted
Precision: 0.9828591235982194
Recall: 0.9279429370348355
F1 Score: 0.9278196686351913

Mean FNR: 0.6272750537705691
Mean TNR: 0.9704148340774352
Mean FPR: 0.029585165922564816
Mean TPR: 0.3727249462294309


In [15]:
attack = FastGradientMethod(estimator=classifier, eps=0.25)

x_test_adv = attack.generate(x=x_test)

In [16]:
calculate_performance_metrics(x_test_adv, y_test, model)

7466/7466 [==============================] - 4s 555us/step
Accuracy: 0.9278466600249483
AUC: 0.8113032692884008

macro
Precision: 0.4803689565234757
Recall: 0.3488991599802701
F1 Score: 0.3241727589711294

weighted
Precision: 0.9829418368713307
Recall: 0.9278466600249483
F1 Score: 0.9285326268581412

Mean FNR: 0.6511008400197299
Mean TNR: 0.9746986807152286
Mean FPR: 0.025301319284771313
Mean TPR: 0.3488991599802701


In [17]:
attack = FastGradientMethod(estimator=classifier, eps=0.3)

x_test_adv = attack.generate(x=x_test)

In [18]:
calculate_performance_metrics(x_test_adv, y_test, model)

7466/7466 [==============================] - 4s 552us/step
Accuracy: 0.9278131723693354
AUC: 0.7692164410016809

macro
Precision: 0.47409935136858294
Recall: 0.34400644255562324
F1 Score: 0.32005742699711126

weighted
Precision: 0.9836235010128992
Recall: 0.9278131723693354
F1 Score: 0.9292890140657181

Mean FNR: 0.6559935574443767
Mean TNR: 0.978731199163829
Mean FPR: 0.02126880083617095
Mean TPR: 0.34400644255562324
